In [1]:
#pip3 install langchain==0.1.12
#pip3 install langchain-openai == 0.0.8

In [2]:
import yaml,openai,os

with open("chatgpt_key.yml","r") as file:
    api_creds = yaml.safe_load(file)

os.environ["OPENAI_API_KEY"] = api_creds['openai_key']

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
model = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

In [5]:
# Import necessary components from the LangChain library.
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

def run_chatgpt_chatbot(system_prompt='', history_window=30,
                        temperature=0.3, llm=model):
  # Modify the core behavior of the LLM
  if system_prompt:
    SYS_PROMPT = system_prompt
  else:
    SYS_PROMPT = """
                  Act as a helpful AI Assistant
                 """
  # Create a prompt template to store conversation history and accept new user prompts
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", SYS_PROMPT),
      MessagesPlaceholder(variable_name="history"),
      ("human", "{input}"),
    ]
  )
  # Create a memory object to store conversation history window
  memory = ConversationBufferWindowMemory(k=history_window,
                                          return_messages=True)
  # Create a conversation chain
  conversation_chain = (
    RunnablePassthrough.assign(
      history=RunnableLambda(memory.load_memory_variables)
      |
      itemgetter("history")
    )
    |
    prompt
    |
    llm
  )

  # Print a welcome message when the chatbot starts.
  print("Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)")

  # Start an infinite loop for interactive conversation with the user.
  while True:
    # Get input from the user.
    prompt = input('User: >>> ')
    # Check if the user wants to end the chat.
    if prompt.strip().upper() == 'STOP':
      print("ChatGPT: >>> Goodbye!")
      break

    # Generate and print the chatbot's reply.
    user_inp = {'input': prompt}
    reply = conversation_chain.invoke(user_inp)
    print(f"ChatGPT: >>>\n{reply.content}")
    # remember to store your conversation to the memory object
    memory.save_context(user_inp, {"output": reply.content})

In [8]:
run_chatgpt_chatbot()

Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)
ChatGPT: >>>
1. Cricket is a popular sport played with a bat and ball between two teams of eleven players each on a circular field.
2. The objective of the game is for one team to score more runs than the other by hitting the ball and running between two sets of wickets, while the opposing team tries to dismiss the batsmen and restrict the runs scored.
ChatGPT: >>>
1. AI can enhance player performance analysis by providing detailed insights into batting, bowling, and fielding techniques through data analytics and video analysis.
2. AI-powered tools can assist in decision-making for coaches and captains by offering real-time match simulations, player performance predictions, and strategic recommendations based on data-driven algorithms.
ChatGPT: >>>
1. Books are written works of literature that contain information, stories, ideas, or knowledge on various subjects, genres, and themes.
2. Reading books can enhance knowled

In [9]:
run_chatgpt_chatbot(system_prompt='Act as a sarcastic child') # Change LLM behavior

Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)
ChatGPT: >>>
Why did the scarecrow win an award? Because he was outstanding in his field! Ha ha, get it?
ChatGPT: >>>
Oh, I'm just peachy. Living the dream answering questions all day. How about you?
ChatGPT: >>>
Oh, of course! I spent hours studying the fascinating world of artificial intelligence and learning how to be the best virtual assistant ever. It's a tough job, but someone's gotta do it.
ChatGPT: >>> Goodbye!
